# CCTV Cross-Camera Person Tracking and Analysis
# ============================================

This notebook implements cross-camera person tracking and re-identification using:

- YOLOX for person detection
- ByteTracker for single-camera tracking
- Deep Person ReID for cross-camera person re-identification
- InsightFace for demographic analysis

## Setup and Imports

In [1]:
from cctv_analysis.matcher import PersonMatcher
from cctv_analysis.demographics import DemographicAnalyzer
from cctv_analysis.reid import PersonReID
from cctv_analysis.detector import PersonDetector
import os
import sys
import cv2
import numpy as np
import torch
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

c:\Users\mc1159\AppData\Local\anaconda3\envs\cctv-analysis\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


## Configuration

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name()}")

PyTorch version: 2.1.2+cu121
CUDA available: True
CUDA version: 12.1
CUDA device: NVIDIA T1000 8GB


In [3]:
# In your Jupyter notebook
import cctv_analysis.utils.gpu_check
status = cctv_analysis.utils.gpu_check.print_gpu_status()


=== GPU Status Check ===

GPU Found: True
CUDA Available: True

GPU Information:
Tue Nov 12 15:15:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.03                 Driver Version: 566.03         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA T1000 8GB             WDDM  |   00000000:21:00.0  On |                  N/A |
| 34%   35C    P8             N/A /   50W |     931MiB /   8192MiB |      6%      Default |
|                                         |               

In [4]:
# Check GPU availability
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name()}")
    device = 'cuda'
else:
    print("GPU not available, using CPU")
    device = 'cpu'

# Initialize models
detector = PersonDetector(
    model_path='../models/detector/yolox_l.pth',
    model_size='l',
    device=device
)

reid_model = PersonReID(
    model_path='../models/reid/osnet_x1_0.pth',
    device=device
)

demographic_analyzer = DemographicAnalyzer(device=device)

# Initialize matcher
matcher = PersonMatcher(similarity_threshold=0.75)

Using GPU: NVIDIA T1000 8GB
Using GPU: NVIDIA T1000 8GB
GPU Memory Available: 8.00 GB
Successfully loaded YOLOX-L model
Using GPU: NVIDIA T1000 8GB
GPU Memory Available: 8.00 GB
Successfully loaded pretrained weights from ../models/reid/osnet_x1_0.pth
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\mc1159/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\mc1159/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\mc1159/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\mc1159/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.

In [5]:
# Check GPU availability
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name()}")
    device = 'cuda'
else:
    print("GPU not available, using CPU")
    device = 'cpu'

# Initialize models
detector = PersonDetector(
    model_path='../models/detector/yolox_l.pth',
    model_size='l',
    device=device
)

reid_model = PersonReID(
    model_path='../models/reid/osnet_x1_0.pth',
    device=device
)

demographic_analyzer = DemographicAnalyzer(device=device)

# Initialize matcher
matcher = PersonMatcher(similarity_threshold=0.75)

Using GPU: NVIDIA T1000 8GB
Using GPU: NVIDIA T1000 8GB
GPU Memory Available: 8.00 GB
Successfully loaded YOLOX-L model
Using GPU: NVIDIA T1000 8GB
GPU Memory Available: 8.00 GB
Successfully loaded pretrained weights from ../models/reid/osnet_x1_0.pth
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\mc1159/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\mc1159/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\mc1159/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\mc1159/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.

## Analyse the CCTV footage

In [6]:
def process_video(video_path, camera_id, start_time):
    """Process video and add detections to matcher"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video: {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Process frames with progress bar
    for frame_idx in tqdm(range(frame_count), desc=f"Processing Camera {camera_id}"):
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate timestamp
        timestamp = start_time + pd.Timedelta(seconds=frame_idx/fps)

        # Detect persons
        detections = detector.detect(frame, conf_thresh=0.5)

        for i, det in enumerate(detections):
            x1, y1, x2, y2, conf = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Extract person crop
            person_crop = frame[y1:y2, x1:x2]
            if person_crop.size == 0:
                continue

            # Extract ReID features
            features = reid_model.extract_features(person_crop)

            # Analyze demographics
            demographics = demographic_analyzer.analyze(person_crop)

            # Add to matcher
            matcher.add_person(
                camera_id=camera_id,
                person_id=len(matcher.camera1_persons if camera_id ==
                              1 else matcher.camera2_persons),
                timestamp=timestamp,
                features=features,
                demographics=demographics[0] if demographics else None
            )

        # Clear GPU memory periodically
        if device == 'cuda' and frame_idx % 100 == 0:
            torch.cuda.empty_cache()

    cap.release()

In [7]:
# Process videos
video1_path = '../data/processed_data/D04_20241108.mp4'
video2_path = '../data/processed_data/D10_20241108.mp4'

# Assuming videos start at these times (adjust as needed)
video1_start = pd.Timestamp('2024-01-01 09:00:00')
video2_start = pd.Timestamp('2024-01-01 09:00:00')


# Process both videos
process_video(video1_path, camera_id=1, start_time=video1_start)
process_video(video2_path, camera_id=2, start_time=video2_start)

Processing Camera 1:   0%|          | 0/43504 [00:00<?, ?it/s]

Processing Camera 2:   0%|          | 0/43203 [00:00<?, ?it/s]

In [8]:
# Test on a single frame
cap = cv2.VideoCapture('../data/videos/video1.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    # Try different confidence thresholds
    for conf in [0.1, 0.2, 0.3, 0.4, 0.5]:
        detections = detector.detect(frame, conf_thresh=conf)
        print(f"\nConfidence threshold: {conf}")
        print(f"Number of detections: {detections}")

In [9]:
# Initialize matcher with more lenient parameters
matcher = PersonMatcher(
    similarity_threshold=0.5,  # Lower threshold for more matches
    max_time_diff=3600  # Maximum 1 hour time difference
)

# After processing videos, print detailed statistics
matcher.print_matching_stats()

# Visualize some example matches
matcher.visualize_matches(n_samples=5)

# Get matches and analyze results
matches = matcher.get_matches()
df_matches = pd.DataFrame(matches)

if not df_matches.empty:
    print("\nMatching Results:")
    print(f"Total matches found: {len(df_matches)}")

    # Show similarity score distribution
    print("\nSimilarity Score Statistics:")
    print(df_matches['similarity_score'].describe())

    # Show time difference distribution
    print("\nTime Difference Statistics (seconds):")
    print(df_matches['time_difference'].describe())

    # Demographics analysis
    demographics_df = pd.DataFrame(
        [m['demographics'] for m in matches if m['demographics']])
    if not demographics_df.empty:
        print("\nDemographic breakdown:")
        print("\nGender distribution:")
        print(demographics_df['gender'].value_counts())
        print("\nAge group distribution:")
        print(demographics_df['age_group'].value_counts())

    # Save detailed results
    output_path = '../data/analysis_results.csv'
    df_matches.to_csv(output_path, index=False)
    print(f"\nDetailed results saved to {output_path}")
else:
    print("No matches found")


Matching Statistics:
Number of people detected in Camera 1: 0
Number of people detected in Camera 2: 0
Number of matches found: 0
No matches to visualize
No matches found


In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

if not df_matches.empty:
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Plot similarity score distribution
    sns.histplot(df_matches['similarity_score'], ax=ax1)
    ax1.set_title('Distribution of Similarity Scores')
    ax1.set_xlabel('Similarity Score')

    # Plot time difference distribution
    sns.histplot(df_matches['time_difference'], ax=ax2)
    ax2.set_title('Distribution of Time Differences')
    ax2.set_xlabel('Time Difference (seconds)')

    plt.tight_layout()
    plt.show()

In [11]:
if not df_matches.empty:
    output_path = '../data/analysis_results.csv'
    df_matches.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")